# Multiple Seldon Core Operators

This notebook illustrate how multiple Seldon Core Operators can share the same cluster. In particular:

  * A Namespaced Operator that only manages Seldon Deployments inside its namespace. Only needs Role RBAC and Namespace labeled with `controller-id`
  * A Clusterwide Operator that manges SeldonDeployment with a matching `controller-id` label.
  * A Clusterwide Operator that manages Seldon Deployments not handled by the above.

## Namespaced Seldon Core Operator

In [ ]:
!kubectl create namespace seldon-ns1

In [ ]:
!kubectl label namespace seldon-ns1 controller-id=seldon-ns1

In [ ]:
!helm install seldon-namespaced  ../helm-charts/seldon-core-operator  \
    --set singleNamespace=true \
    --set image.pullPolicy=IfNotPresent \
    --set usageMetrics.enabled=false \
    --set crd.create=true \
    --namespace seldon-ns1

In [ ]:
!kubectl rollout status deployment/seldon-controller-manager -n seldon-ns1

In [ ]:
!kubectl create -f resources/model.yaml -n seldon-ns1

In [ ]:
!kubectl get sdep -n seldon-ns1

## Label Focused Seldon Core Operator

 * We set `crd.create=false` as the CRD already exists in the cluster.
 * We set `controllerId=seldon-id1`. SeldonDeployments with this label will be managed.

In [ ]:
!kubectl create namespace seldon-id1

In [ ]:
!helm install seldon-controllerid  ../helm-charts/seldon-core-operator  \
    --set singleNamespace=false \
    --set image.pullPolicy=IfNotPresent \
    --set usageMetrics.enabled=false \
    --set crd.create=false \
    --set controllerId=seldon-id1 \
    --namespace seldon-id1

In [ ]:
!kubectl rollout status deployment/seldon-controller-manager -n seldon-id1

In [ ]:
!pygmentize resources/model_controller_id.yaml

In [ ]:
!kubectl create -f resources/model_controller_id.yaml -n default

In [ ]:
!kubectl get sdep -n default

## Global Seldon Core Operator

In [ ]:
!kubectl create namespace seldon-clusterwide

In [ ]:
!helm install seldon-clusterwide  ../helm-charts/seldon-core-operator  \
    --set singleNamespace=false \
    --set image.pullPolicy=IfNotPresent \
    --set usageMetrics.enabled=false \
    --set crd.create=false \
    --namespace seldon-clusterwide

In [ ]:
!kubectl rollout status deployment/seldon-controller-manager -n seldon-clusterwide

Create a model in default and check its running

In [ ]:
!kubectl create -f resources/model.yaml -n default

In [ ]:
!kubectl get sdep -n default